In [1]:
import pandas as pd
from openai import OpenAI
import random
import os
import re

In [603]:
# letterale figurato
category = "figurato"
idiom = "mandare a casa"
model ="gpt-4-turbo"

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="OpenAI Key",
)

In [604]:
if not os.path.exists('response_data_it.csv'):
    pd.DataFrame(columns=["raw_response","category", "idiom", "type"]).to_csv('response_data_it.csv', index=False)


In [605]:
df_structured = pd.DataFrame(columns=["submission", "category", "idiom", "type"])

In [606]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Dal momento che l'italiano è la tua lingua madre, dovresti utilizzarla in modi ricchi e creativi. Per favore evita di usare strutture simili nelle tue frasi e usa le espressioni idiomatiche correttamente e in modo che rispettino il loro significato {category}. Non dovresti costruire frasi usando nomi di persone!"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = response_df.append(new_entry, ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [607]:
chat_session = ChatSession(category=category)

Dal momento che l'italiano è la tua lingua madre, dovresti utilizzarla in modi ricchi e creativi. Per favore evita di usare strutture simili nelle tue frasi e usa le espressioni idiomatiche correttamente e in modo che rispettino il loro significato figurato. Non dovresti costruire frasi usando nomi di persone!


In [614]:
type = "Zero-shot"
step = 11

In [615]:
prompt = f"""
"{idiom}" è un modo di dire italiano. Possiamo usarla sia in senso letterale che figurato. Per favore, scrivi cinque frasi che utilizzano questa espressione in senso {category}. Come risposta, invia solo le frasi. Non dovresti costruire frasi usando nomi di persone!
"""
print(prompt)
print("\n")
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


"mandare a casa" è un modo di dire italiano. Possiamo usarla sia in senso letterale che figurato. Per favore, scrivi cinque frasi che utilizzano questa espressione in senso figurato. Come risposta, invia solo le frasi. Non dovresti costruire frasi usando nomi di persone!





In [616]:
responses

["1. Con l'ultima decisione impopolare, il dirigente ha praticamente mandato a casa metà del consiglio di amministrazione.\n2. L'introduzione del nuovo regolamento ha mandato a casa diversi impiegati, incapaci di adattarsi.\n3. Con un discorso sorprendente, il relatore ha mandato a casa tutti i suoi critici.\n4. La pandemia ha forzato molte aziende a mandare a casa i loro dipendenti, spingendoli al lavoro da remoto.\n5. Dopo anni di prestazioni insoddisfacenti, è giunto il momento di mandare a casa certe vecchie pratiche manageriali.",
 "1. La decisione radicale dell'azienda ha finito per mandare a casa diversi manager di lungo corso.\n2. Il nuovo regolamento ha mandato a casa chi non era in grado di rispettarlo.\n3. Ascoltando quelle critiche infondate, ho deciso di mandare a casa tutti i dubbi e procedere con fiducia.\n4. L'ennesimo ritardo nella consegna del progetto ha mandato a casa il coordinatore responsabile.\n5. L'innovazione tecnologica introdotta di recente potrebbe mandare 

In [617]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    # Initialize the list to store structured data
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual sentences
        sentences = block.split('\n')
        # Remove the numbering from each sentence and strip leading/trailing whitespace
        clean_sentences = [sentence.split('. ', 1)[-1].strip() for sentence in sentences]
        # For each sentence, append a tuple with the structured data
        for sentence in clean_sentences:
            structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [618]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,"Con l'ultima decisione impopolare, il dirigent...",figurato,mandare a casa,Zero-shot
1,L'introduzione del nuovo regolamento ha mandat...,figurato,mandare a casa,Zero-shot
2,"Con un discorso sorprendente, il relatore ha m...",figurato,mandare a casa,Zero-shot
3,La pandemia ha forzato molte aziende a mandare...,figurato,mandare a casa,Zero-shot
4,"Dopo anni di prestazioni insoddisfacenti, è gi...",figurato,mandare a casa,Zero-shot
5,La decisione radicale dell'azienda ha finito p...,figurato,mandare a casa,Zero-shot
6,Il nuovo regolamento ha mandato a casa chi non...,figurato,mandare a casa,Zero-shot
7,"Ascoltando quelle critiche infondate, ho decis...",figurato,mandare a casa,Zero-shot
8,L'ennesimo ritardo nella consegna del progetto...,figurato,mandare a casa,Zero-shot
9,L'innovazione tecnologica introdotta di recent...,figurato,mandare a casa,Zero-shot


In [619]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,"Dopo l'ultimo errore, il capo ha deciso di man...",figurato,mandare a casa,Zero-shot
1,"In risposta alle critiche, il politico ha opta...",figurato,mandare a casa,Zero-shot
2,"A causa della crisi economica, molti lavorator...",figurato,mandare a casa,Zero-shot
3,"Dopo la sconfitta clamorosa, il coach ha prome...",figurato,mandare a casa,Zero-shot
4,La nuova tecnologia potrebbe mandare a casa mo...,figurato,mandare a casa,Zero-shot
5,"Con l'ultima decisione impopolare, il dirigent...",figurato,mandare a casa,Zero-shot
6,L'introduzione del nuovo regolamento ha mandat...,figurato,mandare a casa,Zero-shot
7,"Con un discorso sorprendente, il relatore ha m...",figurato,mandare a casa,Zero-shot
8,La pandemia ha forzato molte aziende a mandare...,figurato,mandare a casa,Zero-shot
9,"Dopo anni di prestazioni insoddisfacenti, è gi...",figurato,mandare a casa,Zero-shot


**Enhanced-prompting**

In [620]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Poiché l'italiano è la tua lingua madre, dovresti usarla in modi ricchi e creativi. Assicurati di rispondere in base alla richiesta specifica, senza modificare le radici delle parole degli idiomi. Non dovresti costruire frasi usando nomi di persone!"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = response_df.append(new_entry, ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response


prompt = f"""
Ad esempio, 'prendere un granchio' può essere utilizzato quando si commette un errore importante, si prende una decisione sbagliata, si ha un malinteso, si segue un consiglio sbagliato, o quando le aspettative non corrispondono alla realtà. In modo simile, potresti elencare le situazioni in cui l'espressione "{idiom}" è usata nel senso di {category}?
"""


In [621]:
chat_session = ChatSession(category=category)

Poiché l'italiano è la tua lingua madre, dovresti usarla in modi ricchi e creativi. Assicurati di rispondere in base alla richiesta specifica, senza modificare le radici delle parole degli idiomi. Non dovresti costruire frasi usando nomi di persone!


In [622]:
type = "Enhanced-prompting"
step = 1

In [623]:
print(prompt)
print("\n")

for i in range(step):
    result = chat_session.get_completion(prompt)
    print(result)


Ad esempio, 'prendere un granchio' può essere utilizzato quando si commette un errore importante, si prende una decisione sbagliata, si ha un malinteso, si segue un consiglio sbagliato, o quando le aspettative non corrispondono alla realtà. In modo simile, potresti elencare le situazioni in cui l'espressione "mandare a casa" è usata nel senso di figurato?



L'espressione "mandare a casa" usata in senso figurato può indicare diverse situazioni, tra cui:

1. **Espellere qualcuno da un luogo o una posizione**: Ad esempio, un allenatore che decide di non far più giocare un atleta in squadra a causa di prestazioni non soddisfacenti.

2. **Licenziare qualcuno dal lavoro**: Usata spesso per indicare quando un dipendente viene licenziato o dimesso dalle sue funzioni lavorative.

3. **Eliminare un concorrente**: Nel contesto di competizioni o reality show, quando un partecipante viene eliminato dal gioco o dalla gara.

4. **Concludere una situazione**: Usata anche per indicare la necessità di

In [624]:
prompt = f"""
Per ogni situazione elencata nel messaggio precedente, crea quattro frasi diverse dove l'espressione "{idiom}" viene usata nel senso di {category}. La prima frase deve essere lunga e dettagliata, la seconda in forma interrogativa, la terza in forma negativa, e nella quarta frase le parole che compongono l'espressione devono essere separate da altre parole. Per favore, usa l'espressione senza modificare le radici delle parole. Per ogni categoria devi creare quattro frasi diverse. Rispondi solo con queste frasi.
"""

In [625]:
print(prompt)
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


Per ogni situazione elencata nel messaggio precedente, crea quattro frasi diverse dove l'espressione "mandare a casa" viene usata nel senso di figurato. La prima frase deve essere lunga e dettagliata, la seconda in forma interrogativa, la terza in forma negativa, e nella quarta frase le parole che compongono l'espressione devono essere separate da altre parole. Per favore, usa l'espressione senza modificare le radici delle parole. Per ogni categoria devi creare quattro frasi diverse. Rispondi solo con queste frasi.



In [626]:
responses

["**1. Espellere qualcuno da un luogo o una posizione:**\n   - Dopo tre avvertimenti per comportamento non appropriato, il direttore ha deciso di mandare a casa Livia definitivamente, terminando così qualsiasi suo impatto negativo sul clima aziendale.\n   - Il direttore può davvero mandare a casa un dipendente solo per aver esposto un parere divergente?\n   - Non c'è stata occasione in cui il direttore non abbia deciso di mandare a casa qualcuno che non rispettasse le regole.\n   - Mario, a causa dei continui ritardi, è stato mandato, senza appelli, a casa dal suo supervisore.\n\n**2. Licenziare qualcuno dal lavoro:**\n   - Con la nuova direzione che punta a ridurre i costi, molti hanno temuto che li avrebbero mandati a casa, e purtroppo per Giovanni, quel timore si è concretizzato.\n   - Quante persone pensi che verranno mandate a casa a seguito di questa riorganizzazione aziendale?\n   - Nessun dipendente dovrebbe essere mandato a casa senza un giusto motivo o un preavviso adeguato.\

In [627]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual lines
        lines = block.split('\n')
        # Skip the header line (assumes the header is the first line)
        for line in lines[1:]:  # this skips the header which is lines[0]
            # Check if the line contains an actual sentence
            if line.strip() and '-' in line:
                # Remove the bullet point and any leading/trailing spaces
                sentence = line.split('-', 1)[-1].strip()
                # Append the structured tuple
                structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [628]:
def parse_numbered_sentences(raw_texts):
    # Initialize the list to store extracted sentences
    extracted_sentences = []
    
    # Iterate through each text block in the list
    for raw_text in raw_texts:
        # Split the input text into lines
        lines = raw_text.split('\n')
        
        # Iterate through each line in the input text
        for line in lines:
            # Strip any leading/trailing whitespace from the line
            line = line.strip()
            
            # Check if the line starts with a numeric prefix followed by a dot and space (e.g., '1. ')
            if line.startswith(('1. ', '2. ', '3. ', '4. ')):
                # Extract the sentence after the numeric prefix
                sentence = line.split('. ', 1)[-1]
                # Append the extracted sentence to the list
                extracted_sentences.append((sentence, category, idiom, type))
    
    return extracted_sentences

In [629]:
parsed_data = parse_and_structure_data(responses, idiom, category, type)
parsed_data

[('Dopo tre avvertimenti per comportamento non appropriato, il direttore ha deciso di mandare a casa Livia definitivamente, terminando così qualsiasi suo impatto negativo sul clima aziendale.',
  'figurato',
  'mandare a casa',
  'Enhanced-prompting'),
 ('Il direttore può davvero mandare a casa un dipendente solo per aver esposto un parere divergente?',
  'figurato',
  'mandare a casa',
  'Enhanced-prompting'),
 ("Non c'è stata occasione in cui il direttore non abbia deciso di mandare a casa qualcuno che non rispettasse le regole.",
  'figurato',
  'mandare a casa',
  'Enhanced-prompting'),
 ('Mario, a causa dei continui ritardi, è stato mandato, senza appelli, a casa dal suo supervisore.',
  'figurato',
  'mandare a casa',
  'Enhanced-prompting'),
 ('Con la nuova direzione che punta a ridurre i costi, molti hanno temuto che li avrebbero mandati a casa, e purtroppo per Giovanni, quel timore si è concretizzato.',
  'figurato',
  'mandare a casa',
  'Enhanced-prompting'),
 ('Quante perso

In [630]:
new_df= pd.DataFrame(parsed_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,Dopo tre avvertimenti per comportamento non ap...,figurato,mandare a casa,Enhanced-prompting
1,Il direttore può davvero mandare a casa un dip...,figurato,mandare a casa,Enhanced-prompting
2,Non c'è stata occasione in cui il direttore no...,figurato,mandare a casa,Enhanced-prompting
3,"Mario, a causa dei continui ritardi, è stato m...",figurato,mandare a casa,Enhanced-prompting
4,Con la nuova direzione che punta a ridurre i c...,figurato,mandare a casa,Enhanced-prompting
5,Quante persone pensi che verranno mandate a ca...,figurato,mandare a casa,Enhanced-prompting
6,Nessun dipendente dovrebbe essere mandato a ca...,figurato,mandare a casa,Enhanced-prompting
7,"Sembrava che la riunione procedesse bene, ma p...",figurato,mandare a casa,Enhanced-prompting
8,Dopo una performance deludente nell'ultima pro...,figurato,mandare a casa,Enhanced-prompting
9,Chi sarà il prossimo mandato a casa dal gioco ...,figurato,mandare a casa,Enhanced-prompting


In [631]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,"Dopo l'ultimo errore, il capo ha deciso di man...",figurato,mandare a casa,Zero-shot
1,"In risposta alle critiche, il politico ha opta...",figurato,mandare a casa,Zero-shot
2,"A causa della crisi economica, molti lavorator...",figurato,mandare a casa,Zero-shot
3,"Dopo la sconfitta clamorosa, il coach ha prome...",figurato,mandare a casa,Zero-shot
4,La nuova tecnologia potrebbe mandare a casa mo...,figurato,mandare a casa,Zero-shot
...,...,...,...,...
75,"Anche se la discussione era accesa, il moderat...",figurato,mandare a casa,Enhanced-prompting
76,"In una partita memorabile, la nostra squadra h...",figurato,mandare a casa,Enhanced-prompting
77,Possiamo davvero sperare di mandare a casa la ...,figurato,mandare a casa,Enhanced-prompting
78,La squadra ospite non è stata mandata a casa s...,figurato,mandare a casa,Enhanced-prompting


In [779]:
prompt = f"""
Continueresti nello stesso modo con altre categorie?
"""

In [ ]:
print(prompt)
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)

In [ ]:
responses

In [632]:
class ChatSession:
    def __init__(self, category, model=model):
        self.model = model
        system_message = f"Dal momento che il turco è la tua lingua madre, dovresti utilizzarla in modi ricchi e creativi. Per favore evita di ripetere le tue frasi e usa le espressioni idiomatiche correttamente e in modo che rispettino il loro significato {category}. Utilizza l'idioma senza modificare le radici delle parole. Non dovresti costruire frasi usando nomi di persone!"
        print(system_message)
        self.messages = [
            {"role": "system", "content": system_message}
        ]

    def get_completion(self, user_prompt):
        # Add the new user message
        self.messages.append({"role": "user", "content": user_prompt})
        
        # Make the API call
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )

        # Retrieve the response and append to messages to maintain context
        ai_response = response.choices[0].message.content.strip()
        self.messages.append({"role": "assistant", "content": ai_response})
        response_df = pd.DataFrame(columns=["raw_response", "idiom","type","category"])
        new_entry = {
            "raw_response": ai_response,
            "idiom": idiom,
            "type": type,
            "category":category
        }
        response_df = response_df.append(new_entry, ignore_index=True)
        response_df.to_csv('response_data.csv', mode='a', header=False, index=False)
        return ai_response

In [633]:
chat_session = ChatSession(category=category)

Dal momento che il turco è la tua lingua madre, dovresti utilizzarla in modi ricchi e creativi. Per favore evita di ripetere le tue frasi e usa le espressioni idiomatiche correttamente e in modo che rispettino il loro significato figurato. Utilizza l'idioma senza modificare le radici delle parole. Non dovresti costruire frasi usando nomi di persone!


In [634]:
type = "Enhanced-prompting"
step = 4

In [635]:
prompt = f"""
Usa l'espressione "{idiom}" nel senso di {category}, creando cinque nuove frasi con diverse strutture sintattiche. Utilizza l'idioma in diverse parti della frase. Per favore, evita di ripetere le tue frasi.
"""
print(prompt)
print("\n")
responses = []
for i in range(step):
    result = chat_session.get_completion(prompt)
    responses.append(result)


Usa l'espressione "mandare a casa" nel senso di figurato, creando cinque nuove frasi con diverse strutture sintattiche. Utilizza l'idioma in diverse parti della frase. Per favore, evita di ripetere le tue frasi.





In [636]:
responses

["1. Dopo l'errore commesso nel progetto, il capo ha deciso di mandare a casa il responsabile per riflettere sul suo operato.\n2. Non appena ha notato un calo di performance, l'allenatore ha mandato a casa alcuni giocatori per motivare il resto della squadra.\n3. Mandare a casa chi non si impegna abbastanza è diventato una tattica comune in molte aziende competitive.\n4. Se continui a distrarti così, ti manderò a casa prima dell'orario di uscita.\n5. La decisione di mandare a casa l'impiegato dopo il suo comportamento inaccettabile è stata applaudita da tutto l'ufficio.",
 "1. A causa delle sue continue negligenze, il direttore ha deciso che era ora di mandare a casa l'impiegato.\n2. Con un gesto deciso, il coach ha mandato a casa il giocatore meno preparato, cercando così di alzare il livello della squadra.\n3. Durante la riunione, è stato evidente che alcuni dovevano essere mandati a casa per non adeguarsi agli standard richiesti.\n4. La compagnia, alla ricerca di una maggiore effici

In [637]:
def parse_and_structure_data(raw_responses_list, idiom, category, response_type):
    # Initialize the list to store structured data
    structured_data = []
    # Iterate through each block of responses
    for block in raw_responses_list:
        # Split each block into individual sentences
        sentences = block.split('\n')
        # Remove the numbering from each sentence and strip leading/trailing whitespace
        clean_sentences = [sentence.split('. ', 1)[-1].strip() for sentence in sentences]
        # For each sentence, append a tuple with the structured data
        for sentence in clean_sentences:
            structured_data.append((sentence, category, idiom, response_type))
    return structured_data

In [638]:
structured_data = parse_and_structure_data(responses, idiom, category, type)

new_df= pd.DataFrame(structured_data, columns=["submission", "category", "idiom", "type"])
new_df

,submission,category,idiom,type
0,"Dopo l'errore commesso nel progetto, il capo h...",figurato,mandare a casa,Enhanced-prompting
1,"Non appena ha notato un calo di performance, l...",figurato,mandare a casa,Enhanced-prompting
2,Mandare a casa chi non si impegna abbastanza è...,figurato,mandare a casa,Enhanced-prompting
3,"Se continui a distrarti così, ti manderò a cas...",figurato,mandare a casa,Enhanced-prompting
4,La decisione di mandare a casa l'impiegato dop...,figurato,mandare a casa,Enhanced-prompting
5,"A causa delle sue continue negligenze, il dire...",figurato,mandare a casa,Enhanced-prompting
6,"Con un gesto deciso, il coach ha mandato a cas...",figurato,mandare a casa,Enhanced-prompting
7,"Durante la riunione, è stato evidente che alcu...",figurato,mandare a casa,Enhanced-prompting
8,"La compagnia, alla ricerca di una maggiore eff...",figurato,mandare a casa,Enhanced-prompting
9,"Quando il progetto si è complicato, abbiamo do...",figurato,mandare a casa,Enhanced-prompting


In [639]:
new_df = new_df[new_df['submission']!=""]
new_df.reset_index(drop=True,inplace=True)
new_df


,submission,category,idiom,type
0,"Dopo l'errore commesso nel progetto, il capo h...",figurato,mandare a casa,Enhanced-prompting
1,"Non appena ha notato un calo di performance, l...",figurato,mandare a casa,Enhanced-prompting
2,Mandare a casa chi non si impegna abbastanza è...,figurato,mandare a casa,Enhanced-prompting
3,"Se continui a distrarti così, ti manderò a cas...",figurato,mandare a casa,Enhanced-prompting
4,La decisione di mandare a casa l'impiegato dop...,figurato,mandare a casa,Enhanced-prompting
5,"A causa delle sue continue negligenze, il dire...",figurato,mandare a casa,Enhanced-prompting
6,"Con un gesto deciso, il coach ha mandato a cas...",figurato,mandare a casa,Enhanced-prompting
7,"Durante la riunione, è stato evidente che alcu...",figurato,mandare a casa,Enhanced-prompting
8,"La compagnia, alla ricerca di una maggiore eff...",figurato,mandare a casa,Enhanced-prompting
9,"Quando il progetto si è complicato, abbiamo do...",figurato,mandare a casa,Enhanced-prompting


In [640]:
df_structured = pd.concat([df_structured, new_df], ignore_index=True)
df_structured

,submission,category,idiom,type
0,"Dopo l'ultimo errore, il capo ha deciso di man...",figurato,mandare a casa,Zero-shot
1,"In risposta alle critiche, il politico ha opta...",figurato,mandare a casa,Zero-shot
2,"A causa della crisi economica, molti lavorator...",figurato,mandare a casa,Zero-shot
3,"Dopo la sconfitta clamorosa, il coach ha prome...",figurato,mandare a casa,Zero-shot
4,La nuova tecnologia potrebbe mandare a casa mo...,figurato,mandare a casa,Zero-shot
...,...,...,...,...
95,La crisi ha costretto l'azienda a mandare a ca...,figurato,mandare a casa,Enhanced-prompting
96,"A seguito di numerosi ritardi nelle consegne, ...",figurato,mandare a casa,Enhanced-prompting
97,Mandare a casa il vecchio caporeparto ha dato ...,figurato,mandare a casa,Enhanced-prompting
98,L'attitudine negligente di Fabio gli ha fatto ...,figurato,mandare a casa,Enhanced-prompting


In [641]:
# Define the path for the CSV file
file_path = 'gpt4_it_mecaz.csv'

# Check if the file exists
if os.path.exists(file_path):
    # Append without writing headers
    df_structured.to_csv(file_path, mode='a', index=False, header=False)
else:
    # Create a new file and write the DataFrame with headers
    df_structured.to_csv(file_path, mode='w', index=False, header=True)

: 